In [4]:
!pip install openai


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import json
import minsearch

with open("documents.json","r") as docs:
    documents_raw = json.load(docs)

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [6]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [7]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    # required but ignored
    api_key='ollama',
)

In [8]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [9]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [11]:
rag("What is data engineering")

" According to the original context provided from a FAQ database:\n\nData engineering is an integral part of the process that involves transforming raw data into information which can be used for analytics purposes such as machine learning. While it was not explicitly described in detail within this specific set of questions and answers, these modules indicate activities related to processing and working with real-time streaming data using Apache Kafka (Module 6) and preparing datasets through Diff Tool's database tools like dbt for analytics engineering (Module 4). Additionally, the context references various technical troubleshooting inquiries regarding Docker setup issues (Module 1), GitHub repo inclusion problems with Git (Module 2), as well as permission errors encountered when working within directories."

In [12]:
print(_)

 According to the original context provided from a FAQ database:

Data engineering is an integral part of the process that involves transforming raw data into information which can be used for analytics purposes such as machine learning. While it was not explicitly described in detail within this specific set of questions and answers, these modules indicate activities related to processing and working with real-time streaming data using Apache Kafka (Module 6) and preparing datasets through Diff Tool's database tools like dbt for analytics engineering (Module 4). Additionally, the context references various technical troubleshooting inquiries regarding Docker setup issues (Module 1), GitHub repo inclusion problems with Git (Module 2), as well as permission errors encountered when working within directories.
